In [3]:
import matplotlib.pyplot as plt
from src import logger
from src.utils.common import read_yaml, log_error, create_directory
from dataclasses import dataclass
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
import cv2
import numpy as np
import os
from typing import List, Union, Optional
import sys
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
print(os.getcwd())


2025-01-14 16:51:00,401 - matplotlib - DEBUG - matplotlib data path: /Users/goldyrana/miniconda3/envs/blood_group/lib/python3.8/site-packages/matplotlib/mpl-data
2025-01-14 16:51:00,405 - matplotlib - DEBUG - CONFIGDIR=/Users/goldyrana/.matplotlib
2025-01-14 16:51:00,407 - matplotlib - DEBUG - interactive is False
2025-01-14 16:51:00,407 - matplotlib - DEBUG - platform is darwin
2025-01-14 16:51:00,449 - matplotlib - DEBUG - CACHEDIR=/Users/goldyrana/.matplotlib
2025-01-14 16:51:00,451 - matplotlib.font_manager - DEBUG - Using fontManager instance from /Users/goldyrana/.matplotlib/fontlist-v330.json


/Users/goldyrana/mess/deep_learning/projects/blood_group_detection/notebooks


In [4]:
os.chdir("../")

In [21]:
@dataclass
class ImagePreprocessingConfig:
    raw_path: str
    interim_path: str

class ConfigurationManager:
    def __init__(self):
        self.config=read_yaml(CONFIG_FILE_PATH)
        self.params=read_yaml(PARAMS_FILE_PATH)
        print(self.config)
    def get_image_preprocessing_config(self):
        params=ImagePreprocessingConfig(
            raw_path=self.config.data.extract_to,
            interim_path=self.config.data.interim_path
        )
        return params



class ImagePreprocessing:
    def __init__(self, config):
        self.config=config

    @log_error(faliure_message="Image Preprocessing")
    def process_image(self, image):
        # corrected_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # corrected_image = corrected_image.astype(np.uint8)

        clahe = cv2.createCLAHE(clipLimit=70.0, tileGridSize=(8, 8))
        enhanced_image = clahe.apply(image)
        
        # Apply erosion to thin and darken lines
        kernel = np.ones((1, 1), np.uint8)
        thinned_image = cv2.erode(enhanced_image, kernel, iterations=200)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(5,5))
        enhanced_contrast = clahe.apply(thinned_image)

        smoothed = cv2.GaussianBlur(enhanced_contrast, (3, 3), 23)
        # Step 3: Sharpen the image
        sharpened = cv2.addWeighted(enhanced_contrast,2, smoothed, -0.5, 0)

        denoised_image = cv2.fastNlMeansDenoising(sharpened, None, h=20, 
                                                templateWindowSize=5, 
                                                searchWindowSize=25)
        kernel = np.ones((1, 1), np.uint8)
        erode_image = cv2.erode(denoised_image, kernel, iterations=200)
        return erode_image
    
    @log_error(faliure_message="Reading Image")
    def read_image(self, path):
        image = cv2.imread(path, cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(np.uint8)
        return image
    
    @log_error(sucess_message=None, faliure_message="Get folder path")
    def get_folder_path(self, main_folder: str) -> List[str]:
        """
        To get sub folder names of given folder 
        Return:
        list: 
        """
        folder_names = list(os.listdir(main_folder)) # returns the folder name
        print(folder_names)
        folder_with_path = [] # to save foldername with base path
        
        for sub_folder in folder_names:
            if os.path.isdir(os.path.join(main_folder, sub_folder)) != True:
                logger.info(f"{sub_folder} not a directory\nRemoving {sub_folder}") # to remove reduant files 
            else:
                folder_with_path.append(os.path.join(main_folder, # base path
                                                     sub_folder)) # image_folder_name
        return folder_with_path

    @log_error(faliure_message="Failed in saving image")
    def save_image(self, image, save_path) -> None:
        create_directory(save_path, is_extension_present=True)
        cv2.imwrite(save_path, image)
        

    def apply_image_operations(self, image_path) -> None:
        try:
            # Read, process, and save the image
            image = self.read_image(image_path)
            image = self.process_image(image)
            self.save_image(image, image_path.replace("raw", "interim"))
            
        except Exception as e:
            return f"Error processing {image_path}: {e}"

    @log_error(faliure_message="Directory not found")
    def get_all_image_path(self, folder_path: List[str]):
        """
        To get the base image path with image_name
        Args:
            folder_path(List[Str]): List containing sub directory names
        Returns:
            all_image_path: List of str; image_path/image_name.extension
        """
        all_image_path = []
        for folder in folder_path:
            image_names = os.listdir(folder)
            for image_name in image_names:
                image_path = os.path.join(folder, image_name)
                all_image_path.append(image_path)
        return all_image_path

def main(func,all_image_path):
    """
    Apply func on all the images using multiprocessing
    Returns None
    """
    with Pool(processes=cpu_count()) as pool:
        results = list(tqdm(pool.imap(func, all_image_path), total=len(all_image_path)))

        

In [22]:
if __name__ == "__main__":
    config_manager = ConfigurationManager()
    config_params = config_manager.get_image_preprocessing_config()

    ob = ImagePreprocessing(config_params)
    folder_path = ob.get_folder_path(config_params.raw_path) #-> raw folder with sub directory

    all_image_path = ob.get_all_image_path(folder_path) # image name with path
    main(ob.apply_image_operations, all_image_path)
    

2025-01-14 15:32:38,749 - root - INFO - Yaml read successfully from config/config.yaml
2025-01-14 15:32:38,750 - root - INFO - Yaml read successfully from params.yaml
2025-01-14 15:32:38,751 - root - ERROR - Exception occured while reading yaml file from                         location: params.yaml
 First argument must be mapping or iterable
2025-01-14 15:32:38,752 - root - INFO - .gitkeep not a directory
Removing .gitkeep
2025-01-14 15:32:38,752 - root - INFO - .DS_Store not a directory
Removing .DS_Store


{'data': {'google_drive_url': 'https://drive.google.com/file/d/10ApjOcTo6tjO34q5et-ij-tbrSm0Waqc/view?usp=drive_link', 'download_path': 'data/external/data.zip', 'extract_to': 'data/raw', 'interim_path': 'data/interim', 'processed_path': 'data/processed'}}
['.gitkeep', 'AB+', 'AB-', 'A+', 'B-', 'B+', 'A-', 'O-', 'O+']
['.DS_Store', 'AB+', 'AB-', 'A+', 'B-', 'B+', 'A-', 'O-', 'O+']


# Src/data/split.py

In [13]:
from dataclasses import dataclass
from src.utils.common import split_image_folder

@dataclass
class SplitFolderConfig:
    interim_path: str
    processed_path: str
    size_ratio: tuple

class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH) 

    def get_split_folder_config(self):
        params = SplitFolderConfig(
            interim_path=self.config.data.interim_path,
            processed_path=self.config.data.processed_path,
            size_ratio=(self.config.data.train_size,
                        self.config.data.test_size,
                        self.config.data.val_size)          
        )
        return params
    
    
class SplitFolder:
    def __init__(self, config):
        self.config=config

    def create_train_test(self):
        split_image_folder(self.config.size_ratio, base_dir=self.config.interim_path,
                           target_dir=self.config.processed_path)
        

if __name__ == "__main__":
    config_manager = ConfigurationManager()
    config_params = config_manager.get_split_folder_config()

    ob = SplitFolder(config_params)
    ob.create_train_test()